In [1]:
!pwd
!pip install -U -r requirements.txt

/examples/qm9
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [2]:
from dask.distributed import Client

client = Client()

In [3]:
from pathlib import Path
from main import download_qm9_data, make_dataset, data_root

file_path: Path | str = data_root.joinpath("data")
manifest_file: str = "manifest.csv"
if not file_path.exists():
    make_dataset(file_path, manifest_file)
manifest_path = file_path.joinpath(manifest_file)

## Pandas DataFrame

In [4]:
from timeit import default_timer as timer
import pandas as pd
from main import make_fingerprint_feature2

df = pd.read_csv(manifest_path)
    
def pandas_worker(df):
    results = df['smiles'].apply(make_fingerprint_feature2)
    return results
    
t = timer()
results = pandas_worker(df)
et = timer() - t
print(f"elapsed time: {et:.3f} secs")
results

elapsed time: 23.584 secs


0         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                                ...                        
130181    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
130182    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
130183    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
130184    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
130185    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: smiles, Length: 130186, dtype: object

## Dask DataFrame

In [5]:
import dask.dataframe as dd

ddf = dd.read_csv(manifest_path)
ddf = ddf.repartition(npartitions=8)

def dask_worker(ddf):
    
    def mff_wrapper(dfd):
        df = dfd.compute()
        return df['smiles'].apply(make_fingerprint_feature2)

    futures = client.map(mff_wrapper, ddf.to_delayed())
    results = pd.concat(client.gather(futures))
    return results

t = timer()
results = dask_worker(ddf)
et = timer() - t
print(f"elapsed time: {et:.3f} secs")
results

elapsed time: 14.564 secs


0         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                                ...                        
130181    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
130182    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
130183    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
130184    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
130185    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: smiles, Length: 130186, dtype: object

## Test dask in training

In [6]:
from dask_ml.model_selection import train_test_split
import numpy as np
import dask.array as da

X = results
y = ddf.homo.compute()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print(X_train)
print(y_train)

X_train = da.stack(X_train).astype(float)
y_train = da.stack(y_train)
print(X_train)
print(y_train)

618       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
112850    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
19915     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
6799      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
55436     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                                ...                        
93137     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
65197     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
42415     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
126807    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
21548     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: smiles, Length: 117167, dtype: object
618      -0.2455
112850   -0.2518
19915    -0.2857
6799     -0.2509
55436    -0.2494
           ...  
93137    -0.2451
65197    -0.2382
42415    -0.2767
126807   -0.2236
21548    -0.2276
Name: homo, Length: 117167, dtype: float64
dask.array<astype, shape=(117167, 4096), dtype=float64, chunksize=

In [ ]:
import joblib
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR


param_space = {
    'C': np.logspace(-6, 6, 13),
    'gamma': np.logspace(-8, 8, 17),
    'tol': np.logspace(-4, -1, 4),
}

model = SVR(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=3, n_iter=5, verbose=10)

with joblib.parallel_backend('dask'):
    search.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV 1/3; 4/5] START C=0.01, gamma=1e-05, tol=0.001..............................
[CV 1/3; 2/5] START C=100000.0, gamma=0.1, tol=0.001............................
[CV 1/3; 5/5] START C=0.1, gamma=0.01, tol=0.1..................................
[CV 1/3; 3/5] START C=100.0, gamma=1.0, tol=0.1.................................
[CV 1/3; 1/5] START C=1.0, gamma=100000000.0, tol=0.1...........................
